In [1]:
# default_exp rank

In [103]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [1]:
# export
import os
import sys
sys.path.append('../mylib/')
from algo.rs.matrix import *
from code.val import main as val
from code.eda import * 
from code.config import args
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import math  
from sklearn.metrics.pairwise import cosine_similarity
from loguru import logger
from utils.pickle import PickleWrapper

In [2]:
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

# val

In [3]:
def val1(df, topk, click_val):
    """
    :sr: pd.Series
        user_id
        1        [52766, 17359, 8142, 114592, 82010, 102105, 11...
        2        [111215, 4207, 49803, 5291, 68704, 113633, 712...
    """
    sr = df.groupby('user_id')['item_id_pred'].agg(list)
    r_list = []
    pred_num = click_val.shape[0]
    for line in click_val.itertuples():
        try:
#             t = [i[0] for i in sr[line.user_id]]
            df.loc[df.user_id==line.user_id, 'item_id_true'] = line.item_id
            t = sr[line.user_id]
#             print(t)
            i = t.index(line.item_id)
        except:
            i = 99999

        r_list.append(i)
    r_series = pd.Series(r_list)
    for i in range(50, topk+1, 50):
        logger.info(f'recall{i}:{r_series[r_series<i].shape[0]/pred_num}')
        
#     r_series[r_series<9999].plot(kind='box', title='pp', rot=45)
    return df

# load click data

In [23]:
all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)

2020-05-27 17:13:45.400 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 0
2020-05-27 17:13:48.786 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-05-27 17:13:48.788 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 1
2020-05-27 17:13:52.711 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-05-27 17:13:52.717 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 2
2020-05-27 17:13:56.730 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-05-27 17:13:56.734 | INFO     | code.eda:load_click_data_per_phase:97 - phase: 3
2020-05-27 17:14:01.287 | INFO     | code.eda:load_click_data_per_phase:108 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-05-27 17:14:01.290 | INFO     | code.ed

In [206]:
item_feat = get_item_data()

/Users/luoyonggui/PycharmProjects/myappsexlib/a_0330_0527_Modern_E_Commerce_Platform_Debiasing/code/eda.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  converters={'item_id':str})


In [208]:
item_feat.iloc[:, 1:129].shape

(108916, 128)

In [209]:
item_feat.iloc[:, 129:257].shape

(108916, 128)

In [211]:
item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

In [212]:
item_feat.set_index('item_id', inplace=True)

In [5]:
phase = [0, 1, 2, 3, 5, 6]
# match_num = 200
match_num = 500

In [185]:
# 计算每个phase的item_id user_cnt  # 貌似每个phase的item只会被user唯一点击一次，那就没必要统计了。。。
def t(s):
    return s.nunique()
all_click_df[0][0].groupby('item_id')['user_id'].agg(t)

item_id
1          4
1000      13
100000     8
10001      5
100013     5
          ..
99990      6
99991      3
99993      5
99994      5
99998      5
Name: user_id, Length: 40775, dtype: int64

In [177]:
# temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
temp_ = all_click_df[0][0].groupby('item_id')['user_id'].agg(list)

In [179]:
temp_

item_id
1                               [1489, 10088, 13420, 19312]
1000      [14546, 9531, 32114, 6934, 20558, 30360, 33492...
100000    [20800, 17064, 33901, 5699, 8541, 1940, 20705,...
10001                      [6572, 17178, 25162, 6383, 8264]
100013                    [18924, 32480, 7696, 6319, 32706]
                                ...                        
99990              [30399, 8084, 2969, 21225, 32152, 34183]
99991                                  [9430, 32852, 31855]
99993                   [23622, 22023, 12546, 31661, 21784]
99994                     [9287, 6305, 28964, 17648, 22420]
99998                    [24217, 9796, 20948, 25551, 16599]
Name: user_id, Length: 40775, dtype: object

In [178]:
temp_.str.len().describe()

count    40775.000000
mean         6.409246
std          6.358793
min          1.000000
25%          4.000000
50%          5.000000
75%          7.000000
max        234.000000
Name: user_id, dtype: float64

In [ ]:
for p in phase:
    # 获取test用户的近n次点击
    temp_ = all_click_df[0][0].groupby('user_id')['item_id'].agg(list)
    temp_ = pd.DataFrame(temp_)
    for i in range(5):
        temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        
    # 计算每个phase的item_id cnt
    all_click_df[0][0].groupby('item_id')['user_id'].count()

In [164]:
whole_click_train, whole_click_val, click_test, test_qtime = load_whole_click_data(args.now_phase, args.DATA_DIR)

2020-05-28 05:06:06.627 | INFO     | code.eda:load_whole_click_data:54 - phase: 0
2020-05-28 05:06:06.989 | INFO     | code.eda:load_whole_click_data:54 - phase: 1
2020-05-28 05:06:07.275 | INFO     | code.eda:load_whole_click_data:54 - phase: 2
2020-05-28 05:06:07.585 | INFO     | code.eda:load_whole_click_data:54 - phase: 3
2020-05-28 05:06:07.939 | INFO     | code.eda:load_whole_click_data:54 - phase: 4
2020-05-28 05:06:08.331 | INFO     | code.eda:load_whole_click_data:54 - phase: 5
2020-05-28 05:06:08.812 | INFO     | code.eda:load_whole_click_data:54 - phase: 6
2020-05-28 05:06:16.617 | INFO     | code.eda:load_whole_click_data:84 - whole_click_train: (954588, 4), whole_click_val: (12081, 4), click_test: (175274, 4), test_qtime: (12081, 3)


In [165]:
click_test_val = click_test.sort_values('time').drop_duplicates('user_id', keep='last')

In [166]:
click_test = click_test[~click_test.index.isin(click_test_val.index)].sort_values('time')

In [167]:
click_test.shape

(109018, 4)

In [168]:
click_test.user_id.nunique()

11724

In [169]:
click_test_val.user_id.nunique()

12081

# load match result

In [76]:
# r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0525.pkl'))

# r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0525.pkl'))

r_itemcf = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527.pkl'))

r_binn = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527.pkl'))

r_itemcf_yl = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_yulao_0527.pkl'))

r_itemcf_yl = [r_itemcf_yl[0], r_itemcf_yl[1],r_itemcf_yl[2],r_itemcf_yl[3],r_itemcf_yl[5],r_itemcf_yl[6]]

In [77]:

df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
for i in range(len(phase)):
    r_itemcf[i]['phase'] = phase[i]
    r_binn[i]['phase'] = phase[i]
    r_itemcf_yl[i]['phase'] = phase[i]
    df1 = df1.append(r_itemcf[i])
    df2 = df2.append(r_binn[i])
    df3 = df3.append(r_itemcf_yl[i])

In [78]:
# s_itemcf.shape, s_binn.shape

# from multiprocessing import Pool
# pool = Pool(6)

In [79]:

# # for i, v in tqdm(s_itemcf.items()):
# def m1(p):
#     i, v = p[0], p[1]
# #     print(i)
#     dft = pd.DataFrame(columns='score1 rank1'.split())
#     cnt = 1
#     for item_id, score in v[:match_num]:
#         dft.loc[item_id] = [score, cnt]
#         cnt += 1
#     dft = dft.reset_index()
#     dft.columns = 'next_item score1 rank1'.split()
#     dft['user_id'] = i
#     return dft
# #     df1 = df1.append(dft)

# dfr_list = pool.map(m1, s_itemcf.items())

# len(dfr_list)


# # for i, v in s_binn.items():
# def m2(p):
#     i, v = p[0], p[1]
# #     print(i)
#     dft = pd.DataFrame(columns='score2 rank2'.split())
#     cnt = 1
#     for item_id, score in v[:match_num]:
#         dft.loc[item_id] = [score, cnt]
#         cnt += 1
#     dft = dft.reset_index()
#     dft.columns = 'next_item score2 rank2'.split()
#     dft['user_id'] = i
#     return dft
# #     df2 = df.append(dft)

# dfr2_list = pool.map(m2, s_binn.items())

# len(dfr2_list)

# df1 = pd.DataFrame()

# for i in tqdm(dfr_list):
#     df1 = df1.append(i)

# df2 = pd.DataFrame()
# for i in tqdm(dfr2_list):
#     df2 = df2.append(i)

# df1.shape

# df2.shape

In [80]:
df1.shape, df2.shape, df3.shape, 

((5186489, 6), (4248843, 6), (4996488, 6))

In [81]:
df1.columns

Index(['user_id', 'item_id_pred', 'score', 'rank', 'item_id_true', 'phase'], dtype='object')

In [139]:

df1_ = df1[df1['rank']<(match_num+1)]
df2_ = df2[df2['rank']<(match_num+1)]
df3_ = df3[df3['rank']<(match_num+1)]

In [140]:
df = pd.merge(pd.merge(df1_, df2_, on='user_id item_id_pred item_id_true phase'.split(), how='outer'), 
              df3_, on='user_id item_id_pred item_id_true phase'.split(), how='outer')

In [141]:
df.shape

(9536065, 10)

In [142]:
df['label'] = (df['item_id_pred'] == df['item_id_true']).map(int)

In [143]:
df.head()

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,0
2,11,10528,5.246875,3.0,8052,0,0.461508,52.0,0.067450,11.0,0
3,11,113564,5.100132,4.0,8052,0,0.386057,66.0,0.136157,1.0,0
4,11,20389,4.636769,5.0,8052,0,0.415892,62.0,0.048790,19.0,0


In [144]:
df.label.value_counts()

0    9533232
1       2833
Name: label, dtype: int64

In [145]:
df.user_id.nunique()

10373

In [146]:
df = df.fillna(0)

In [147]:
df[df.label==1]

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label
101,11,8052,1.900682,102.0,8052,0,0.324958,87.0,0.018896,82.0,1
1039,44,25164,2.845496,40.0,25164,0,0.207407,180.0,0.037454,29.0,1
2017,66,43836,22.465015,18.0,43836,0,1.488082,18.0,0.100061,13.0,1
3512,132,117039,15.770346,13.0,117039,0,0.955082,21.0,0.174193,2.0,1
8675,319,112967,4.998704,176.0,112967,0,0.280866,201.0,0.079765,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...
9485044,28771,30238,0.000000,0.0,30238,6,0.000000,0.0,0.055383,13.0,1
9503352,30696,30780,0.000000,0.0,30780,6,0.000000,0.0,0.000044,245.0,1
9514064,28749,34356,0.000000,0.0,34356,6,0.000000,0.0,0.001915,412.0,1
9515694,2591,18934,0.000000,0.0,18934,6,0.000000,0.0,0.024958,49.0,1


In [198]:
df['item_cnt'] = 0

In [199]:
dft = pd.DataFrame()
for p in tqdm(phase):
    # 获取test用户的近n次点击
    temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
    temp_ = pd.DataFrame(temp_)
    for i in range(3):
        temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
    dft = dft.append(temp_.reset_index())
    # 计算每个phase的item_id cnt
    t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
    df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

100%|██████████| 6/6 [05:30<00:00, 55.10s/it]


In [202]:
df.user_id.nunique()

10373

In [201]:
dft.user_id.nunique()

10373

In [203]:
df = pd.merge(df, dft)

In [221]:
df['item_id_pred_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

df['item_id_pred_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

In [226]:
for c in tqdm('last_1 last_2 last_3'.split()):
    df[f'{c}_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df[f'{c}_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

In [227]:
df

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label,item_cnt,item_id,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0,47,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48..."
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,0,5,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13..."
2,11,10528,5.246875,3.0,8052,0,0.461508,52.0,0.067450,11.0,0,7,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3...","[3.5714921951293945, -3.301568031311035, -1.44...","[-2.7678756713867188, -0.804641842842102, -2.3..."
3,11,113564,5.100132,4.0,8052,0,0.386057,66.0,0.136157,1.0,0,4,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[0.25579115748405457, -1.0630481243133545, 1.9...","[-1.4899662733078003, 3.0152783393859863, 2.34...","[0.25579115748405457, -1.0630481243133545, 1.9...","[-1.4899662733078003, 3.0152783393859863, 2.34...","[0.25579115748405457, -1.0630481243133545, 1.9...","[-1.4899662733078003, 3.0152783393859863, 2.34...","[0.25579115748405457, -1.0630481243133545, 1.9...","[-1.4899662733078003, 3.0152783393859863, 2.34..."
4,11,20389,4.636769,5.0,8052,0,0.415892,62.0,0.048790,19.0,0,7,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[0.1118144690990448, 1.1982476711273191, -1.76...","[-0.27097439765930176, -1.1485528945922852, -2...","[0.1118144690990448, 1.1982476711273191, -1.76...","[-0.27097439765930176, -1.1485528945922852, -2...","[0.1118144690990448, 1.1982476711273191, -1.76...","[-0.27097439765930176, -1.1485528945922852, -2...","[0.1118144690990448, 1.1982476711273191, -1.76...","[-0.27097439765930176, -1.1485528945922852, -2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9536060,35437,59212,0.000000,0.0,114977,6,0.000000,0.0,0.006946,495.0,0,9,"[98124, 70947, 72221, 95778, 73884, 61651, 476...",103047,98965,102018,"[3.9700653553009033, -2.0250101089477544, 1.50...","[-1.927857995033264, 0.3137434422969818, 0.125...","[3.9700653553009033, -2.0250101089477544, 1.50...","[-1.927857995033264, 0.3137434422969818, 0.125...","[3.9700653553009033, -2.0250101089477544, 1.50...","[-1.927857995033264, 0.3137434422969818, 0.125...","[3.9700653553009033, -2.0250101089477544, 1.50...","[-1.927857995033264, 0.3137434422969818, 0.125..."
9536061,35437,96089,0.000000,0.0,114977,6,0.000000,0.0,0.006922,496.0,0,4,"[98124, 70947, 72221, 95778, 73884, 61651, 476...",103047,98965,102018,"[-0.77618008852005, -0.4886056780815125, -0.81...","[0.0693325474858284, 0.5399050712585449, 2.712...","[-0.77618008852005, -0.4886056780815125, -0.81...","[0.0693325474858284, 0.53990

In [ ]:
# df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))

In [6]:
df = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))

# FE

In [7]:
df.shape

(9536065, 24)

In [9]:
df.head(2)

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label,item_cnt,item_id,last_1,last_2,last_3,item_id_pred_text_vec,item_id_pred_img_vec,last_1_text_vec,last_1_img_vec,last_2_text_vec,last_2_img_vec,last_3_text_vec,last_3_img_vec
0,11,79868,6.019197,1.0,8052,0,2.023002,1.0,0.010625,142.0,0,47,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48...","[2.3873836994171143, 1.274107813835144, 0.2217...","[-2.599696159362793, 1.5488953590393066, -1.48..."
1,11,15844,5.328522,2.0,8052,0,0.326349,86.0,0.086894,6.0,0,5,"[88058, 66082, 29755, 78804, 43074, 75757, 233...",28210,64689,20317,"[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13...","[3.9976482391357417, 1.0751028060913086, 0.722...","[-1.5042964220046997, -1.333038091659546, 0.13..."


In [10]:
df['sim1_text'] = df['item_id_pred_text_vec'] * df['last_1_text_vec']

In [ ]:
df['sim1'] = df['sim1'].map(sum)

In [ ]:
df['sim1']

In [11]:
df['sim1']

0          [5.699600928242546, 1.62335072127577, 0.049188...
1          [15.981191443865097, 1.1558460436654059, 0.521...
2          [12.755556499870181, 10.900351465375024, 2.087...
3          [0.0654291162470324, 1.1300713146061412, 3.700...
4          [0.012502475499901244, 1.435797481362044, 3.12...
                                 ...                        
9536060    [15.761418925360488, 4.100665941340596, 2.2717...
9536061    [0.6024555298149927, 0.23873550865349463, 0.66...
9536062    [7.32240680779887, 0.22351263341926492, 8.9388...
9536063    [11.8720610068724, 3.471794804593685, 2.324436...
9536064    [0.20252429081324408, 3.228560558799032, 0.274...
Name: sim1, Length: 9536065, dtype: object

# model

In [149]:
#export
from sklearn.metrics import classification_report

from catboost import Pool, CatBoostClassifier

from sklearn.model_selection import train_test_split

In [127]:
# df.loc[:, 'rank_x'] = df.rank_x.map(lambda x: 999 if x==0 else x)
# df.loc[:, 'rank_y'] = df.rank_x.map(lambda x: 999 if x==0 else x)
# df.loc[:, 'rank'] = df.rank_x.map(lambda x: 999 if x==0 else x)

In [150]:
for i in 'rank score'.split():
#     for ii in [1, 2]:
#         df['f{i}{ii}']
    df[f'{i}1_sub_{i}2'] = df[f'{i}_x']-df[f'{i}_y']
    df[f'{i}1_add_{i}2'] = df[f'{i}_x']+df[f'{i}_y']
    df[f'{i}1_mul_{i}2'] = df[f'{i}_x']*df[f'{i}_y']
    
    df[f'{i}1_sub_{i}3'] = df[f'{i}_x']-df[f'{i}']
    df[f'{i}1_add_{i}3'] = df[f'{i}_x']+df[f'{i}']
    df[f'{i}1_mul_{i}3'] = df[f'{i}_x']*df[f'{i}']
    
    df[f'{i}2_sub_{i}3'] = df[f'{i}_y']-df[f'{i}']
    df[f'{i}2_add_{i}3'] = df[f'{i}_y']+df[f'{i}']
    df[f'{i}2_mul_{i}3'] = df[f'{i}_y']*df[f'{i}']

In [151]:
cols = ['score_x', 'rank_x', 'score_y', 'rank_y','score', 'rank',
'rank1_sub_rank2',
'rank1_add_rank2', 'rank1_mul_rank2', 'score1_sub_score2',
'score1_add_score2', 'score1_mul_score2', 'rank1_sub_rank3',
'rank1_add_rank3', 'rank1_mul_rank3', 'rank2_sub_rank3',
'rank2_add_rank3', 'rank2_mul_rank3', 'score1_sub_score3',
'score1_add_score3', 'score1_mul_score3', 'score2_sub_score3',
'score2_add_score3', 'score2_mul_score3',
]

In [152]:
df['phase'].value_counts()

6    1820995
5    1776590
3    1599358
2    1565472
1    1524428
0    1249222
Name: phase, dtype: int64

In [153]:
df['fscore'] = 0
df['frank'] = 999

In [154]:
X_train, X_test, y_train, y_test = df.loc[df['phase']<6, cols], df.loc[df['phase']==6, cols], df.loc[df['phase']<6, 'label'],\
                                       df.loc[df['phase']==6, 'label']                 

In [155]:
cat_cols = []
train_data = Pool(data=X_train,
                   label=y_train,
                   cat_features=cat_cols)
val_data = Pool(data=X_test, label=y_test, cat_features=cat_cols) 

In [156]:
df.loc[df['phase']==6].user_id.nunique()

1821

In [157]:
df.loc[df['phase']==6].shape

(1820995, 31)

In [161]:
params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'random_seed': 144,
    'custom_metric': 'F1',
    'loss_function': 'Logloss',
    'class_weights': [1, 20],
#     'use_best_model': False
    }
print(params)
model = CatBoostClassifier(**params)


{'iterations': 100, 'learning_rate': 0.1, 'random_seed': 144, 'custom_metric': 'F1', 'loss_function': 'Logloss', 'class_weights': [1, 20]}


In [108]:
# val1(r_itemcf[-1], 100, all_click_df[6][1])
# 2020-05-27 17:39:59.717 | INFO     | __main__:val1:24 - recall50:0.15595826468973092
# 2020-05-27 17:39:59.728 | INFO     | __main__:val1:24 - recall100:0.18231740801757276


In [162]:
r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

2020-05-27 19:22:48.670 | INFO     | __main__:val1:24 - recall50:0.1630971993410214
2020-05-27 19:22:48.672 | INFO     | __main__:val1:24 - recall100:0.19659527732015375


In [159]:
r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

2020-05-27 19:07:59.575 | INFO     | __main__:val1:24 - recall50:0.16090060406370127
2020-05-27 19:07:59.584 | INFO     | __main__:val1:24 - recall100:0.19549697968149368


In [109]:
r = model.fit(train_data, 
#               eval_set=val_data, 
              verbose=False,plot=False)
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

2020-05-27 18:46:49.899 | INFO     | __main__:val1:24 - recall50:0.15760571114772104
2020-05-27 18:46:49.901 | INFO     | __main__:val1:24 - recall100:0.18451400329489293


In [163]:
model.save_model(os.path.join(args.DATA_DIR, 'data_gen/catboost_model500.dump'))

In [160]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

0


2020-05-27 19:09:39.080 | INFO     | __main__:val1:24 - recall50:0.16199890170236134
2020-05-27 19:09:39.081 | INFO     | __main__:val1:24 - recall100:0.1987918725974739


1


2020-05-27 19:10:40.482 | INFO     | __main__:val1:24 - recall50:0.16254805052169138
2020-05-27 19:10:40.483 | INFO     | __main__:val1:24 - recall100:0.1971444261394838


2


...

2020-05-27 19:16:45.345 | INFO     | __main__:val1:24 - recall50:0.16254805052169138
2020-05-27 19:16:45.348 | INFO     | __main__:val1:24 - recall100:0.19384953322350357


8


2020-05-27 19:17:46.970 | INFO     | __main__:val1:24 - recall50:0.16254805052169138
2020-05-27 19:17:46.973 | INFO     | __main__:val1:24 - recall100:0.1916529379461834


9


2020-05-27 19:18:47.498 | INFO     | __main__:val1:24 - recall50:0.16254805052169138
2020-05-27 19:18:47.502 | INFO     | __main__:val1:24 - recall100:0.19384953322350357


In [138]:
for a in range(10):
    print(a)
    r = model.fit(train_data, 
#                   eval_set=val_data, 
                  verbose=False,plot=False, init_model=model)
    df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

    df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

    r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

0


2020-05-27 18:58:02.723 | INFO     | __main__:val1:24 - recall50:0.15760571114772104
2020-05-27 18:58:02.726 | INFO     | __main__:val1:24 - recall100:0.18616144975288304


1


2020-05-27 18:58:33.515 | INFO     | __main__:val1:24 - recall50:0.15760571114772104
2020-05-27 18:58:33.517 | INFO     | __main__:val1:24 - recall100:0.1872597473915431


2


...

2020-05-27 19:01:43.763 | INFO     | __main__:val1:24 - recall50:0.1587040087863811
2020-05-27 19:01:43.767 | INFO     | __main__:val1:24 - recall100:0.18506315211422295


8


2020-05-27 19:02:17.368 | INFO     | __main__:val1:24 - recall50:0.15815485996705106
2020-05-27 19:02:17.372 | INFO     | __main__:val1:24 - recall100:0.18506315211422295


9


2020-05-27 19:02:50.803 | INFO     | __main__:val1:24 - recall50:0.1587040087863811
2020-05-27 19:02:50.807 | INFO     | __main__:val1:24 - recall100:0.18506315211422295


In [247]:
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)

r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

In [227]:
df.loc[df['phase']==6]

,user_id,item_id_pred,score_x,rank_x,item_id_true,phase,score_y,rank_y,score,rank,label,rank1_sub_rank2,rank1_add_rank2,rank1_mul_rank2,rank1_sub_rank3,rank1_add_rank3,rank1_mul_rank3,rank2_sub_rank3,rank2_add_rank3,rank2_mul_rank3,score1_sub_score2,score1_add_score2,score1_mul_score2,score1_sub_score3,score1_add_score3,score1_mul_score3,score2_sub_score3,score2_add_score3,score2_mul_score3,fscore,frank
1710400,17,6322,23.536825,1.0,55640,6,2.233642,1.0,0.000000,0.0,0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,21.303183,25.770466,52.572834,23.536825,23.536825,0.000000,2.233642,2.233642,0.000000,0.356126,1.0
1710401,17,25684,20.098247,2.0,55640,6,1.012120,10.0,0.052460,7.0,0,-8.0,12.0,20.0,-5.0,9.0,14.0,3.0,17.0,70.0,19.086126,21.110367,20.341845,20.045786,20.150707,1.054360,0.959660,1.064581,0.053096,0.218838,3.0
1710402,17,51966,18.027037,3.0,55640,6,1.600439,2.0,0.023134,81.0,0,1.0,5.0,6.0,-78.0,84.0,243.0,-79.0,83.0,162.0,16.426597,19.627476,28.851180,18.003903,18.050171,0.417040,1.577305,1.623574,0.037025,0.150970,6.0
1710403,17,107085,15.216324,4.0,55640,6,0.652856,24.0,0.128767,1.0,0,-20.0,28.0,96.0,3.0,5.0,4.0,23.0,25.0,24.0,14.563469,15.869180,9.934063,15.087558,15.345091,1.959357,0.524089,0.781622,0.084066,0.330119,2.0
1710404,17,31821,14.821069,5.0,55640,6,0.706890,20.0,0.000000,0.0,0,-15.0,25.0,100.0,5.0,5.0,0.0,20.0,20.0,0.0,14.114179,15.527960,10.476869,14.821069,14.821069,0.000000,0.706890,0.706890,0.000000,0.191946,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139627,8795,91479,0.000000,0.0,40240,6,0.000000,0.0,0.009082,196.0,0,0.0,0.0,0.0,-196.0,196.0,0.0,-196.0,196.0,0.0,0.000000,0.000000,0.000000,-0.009082,0.009082,0.000000,-0.009082,0.009082,0.000000,0.019953,366.0
4139628,8795,39758,0.000000,0.0,40240,6,0.000000,0.0,0.009079,197.0,0,0.0,0.0,0.0,-197.0,197.0,0.0,-197.0,197.0,0.0,0.000000,0.000000,0.000000,-0.009079,0.009079,0.000000,-0.009079,0.009079,0.000000,0.019953,367.0
4139629,8795,76959,0.000000,0.0,40240,6,0.000000,0.0,0.008984,198.0,0,0.0,0.0,0.0,-198.0,198.0,0.0,-198.0,198.0,0.0,0.000000,0.000000,0.000000,-0.008984,0.008984,0.000000,-0.008984,0.008984,0.000000,0.019953,368.0
4139630,8795,101735,0.000000,0.0,40240,6,0.000000,0.0,0.008927,199.0,0,0.0,0.0,0.0,-199.0,199.0,0.0,-199.0,199.0,0.0,0.000000,0.000000,0.000000,-0.008927,0.008927,0.000000,-0.008927,0.008927,0.000000,0.019953,369.0


In [81]:
#export
def val(model, df_val, y_val, cat_cols):
    test_data = Pool(data=df_val,
                  cat_features=cat_cols)
    print(pd.Series(model.predict(test_data)).value_counts())
    print(classification_report(y_val, model.predict(test_data)))
    dfr = pd.DataFrame(y_val)
    dfr.columns = ['true_label']
#     dfr['CHANGE_ID'] = df_val['CHANGE_ID']
    y_test_hat = model.predict_proba(test_data)[:, 1]
    dfr['score'] = y_test_hat
    dfr['predict_label'] = model.predict(test_data)
    dfr = dfr.sort_values('score', ascending=False)
    dfr['order'] = range(1, dfr.shape[0] + 1)
    print(dfr[dfr.true_label == 1])
    return dfr

In [102]:
X_test.shape

(208305, 10)

In [101]:
val(model, X_test, y_test, cat_cols)

0    208305
dtype: int64


/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    208100
           1       0.00      0.00      0.00       205

    accuracy                           1.00    208305
   macro avg       0.50      0.50      0.50    208305
weighted avg       1.00      1.00      1.00    208305

         true_label     score  predict_label   order
958400            1  0.471337              0       4
125400            1  0.428180              0      46
168800            1  0.412973              0      93
58800             1  0.409844              0     104
1860400           1  0.394665              0     148
...             ...       ...            ...     ...
265571            1  0.004100              0  179877
974383            1  0.003933              0  182775
1298148           1  0.003745              0  185544
1310507           1  0.003516              0  190538
1989783           1  0.003029              0  195754

[205 rows x 4 columns]


,true_label,score,predict_label,order
241200,0,0.499405,0,1
26000,0,0.477325,0,2
1906400,0,0.471995,0,3
958400,1,0.471337,0,4
602600,0,0.454994,0,5
...,...,...,...,...
178187,0,0.000198,0,208301
855179,0,0.000195,0,208302
289974,0,0.000195,0,208303
1708754,0,0.000190,0,208304


In [17]:
dft.name='item_id'

In [18]:
dft.reset_index()

,index,score1,rank1
0,79868,6.019197,1.0
1,15844,5.328522,2.0
2,10528,5.246875,3.0
3,113564,5.100132,4.0
4,20389,4.636769,5.0
...,...,...,...
495,100781,0.956455,496.0
496,27555,0.955564,497.0
497,34764,0.953188,498.0
498,73319,0.953118,499.0


In [15]:
dft

,score1,rank1
79868,6.019197,1.0
15844,5.328522,2.0
10528,5.246875,3.0
113564,5.100132,4.0
20389,4.636769,5.0
...,...,...
100781,0.956455,496.0
27555,0.955564,497.0
34764,0.953188,498.0
73319,0.953118,499.0
